# Game Analysis Season 2022-23

## Import libs

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from src import algorithms, Data_analysis_tools as Tools, stats_scraper
import joblib
from keras.models import load_model
import warnings
warnings.filterwarnings('ignore')
from IPython.display import display
pd.options.display.max_columns = None
# pd.options.display.max_rows = None

In [2]:
def data_proc_pipeline(pre_pros_name,post_pros_name, **kwrags):
    
    data = stats_scraper.web_scraper(pre_pros_name, training_dataset=True, **kwargs)
    data = algorithms.pre_process_cols(data)
    data = algorithms.hollinger_formula(data)
    data = algorithms.concat_home_away_stats(data)
    data = algorithms.get_dummies(data)
    data = algorithms.feature_eng(data)
    data = algorithms.clean_data(data)
    data = data.sort_values(by='DATE_Home', key=pd.to_datetime, ascending=False)
    data.to_csv(post_pros_name)
    
    return data

## Inputs

In [3]:
kwargs = {'Season': '2022-23',
          'DateFrom': '',
          'DateTo' : ''}

pre_pros_name = 'teams_boxscore_trad_2022_23_first_half.csv'
post_pros_name = 'validation_data_first_half_2022.csv'

## Data Processing

In [4]:
scrape = True

if scrape:
    data = data_proc_pipeline(pre_pros_name,post_pros_name, **kwargs)
else:
    data = pd.read_csv(post_pros_name, index_col=0)
data

,SEASON_YEAR_Home,TEAM_Home,DATE_Home,W/L_Home,MIN_Home,FGM_Home,FGA_Home,FG%_Home,3PM_Home,3PA_Home,3P%_Home,FTM_Home,FTA_Home,FT%_Home,OREB_Home,DREB_Home,REB_Home,AST_Home,TOV_Home,STL_Home,BLK_Home,BLKA_Home,PF_Home,PFD_Home,PTS_Home,+/-_Home,GP_RANK_Home,W_RANK_Home,L_RANK_Home,W_PCT_RANK_Home,MIN_RANK_Home,FGM_RANK_Home,FGA_RANK_Home,FG_PCT_RANK_Home,FG3M_RANK_Home,FG3A_RANK_Home,FG3_PCT_RANK_Home,FTM_RANK_Home,FTA_RANK_Home,FT_PCT_RANK_Home,OREB_RANK_Home,DREB_RANK_Home,REB_RANK_Home,AST_RANK_Home,TOV_RANK_Home,STL_RANK_Home,BLK_RANK_Home,BLKA_RANK_Home,PF_RANK_Home,PFD_RANK_Home,PTS_RANK_Home,PLUS_MINUS_RANK_Home,OPPONENT_Home,VENUE_Home,VALUE_Home,SEASON_YEAR_Away,TEAM_Away,DATE_Away,W/L_Away,MIN_Away,FGM_Away,FGA_Away,FG%_Away,3PM_Away,3PA_Away,3P%_Away,FTM_Away,FTA_Away,FT%_Away,OREB_Away,DREB_Away,REB_Away,AST_Away,TOV_Away,STL_Away,BLK_Away,BLKA_Away,PF_Away,PFD_Away,PTS_Away,+/-_Away,GP_RANK_Away,W_RANK_Away,L_RANK_Away,W_PCT_RANK_Away,MIN_RANK_Away,FGM_RANK_Away,FGA_RANK_Away,FG_PCT_RANK_Away,FG3M_RANK_Away,FG3A_RANK_Away,FG3_PCT_RANK_Away,FTM_RANK_Away,FTA_RANK_Away,FT_PCT_RANK_Away,OREB_RANK_Away,DREB_RANK_Away,REB_RANK_Away,AST_RANK_Away,TOV_RANK_Away,STL_RANK_Away,BLK_RANK_Away,BLKA_RANK_Away,PF_RANK_Away,PFD_RANK_Away,PTS_RANK_Away,PLUS_MINUS_RANK_Away,OPPONENT_Away,VENUE_Away,VALUE_Away,venue_Away,venue_Home,team_ATL,team_BKN,team_BOS,team_CHA,team_CHI,team_CLE,team_DAL,team_DEN,team_DET,team_GSW,team_HOU,team_IND,team_LAC,team_LAL,team_MEM,team_MIA,team_MIL,team_MIN,team_NOP,team_NYK,team_OKC,team_ORL,team_PHI,team_PHX,team_POR,team_SAC,team_SAS,team_TOR,team_UTA,team_WAS,opponent_ATL,opponent_BKN,opponent_BOS,opponent_CHA,opponent_CHI,opponent_CLE,opponent_DAL,opponent_DEN,opponent_DET,opponent_GSW,opponent_HOU,opponent_IND,opponent_LAC,opponent_LAL,opponent_MEM,opponent_MIA,opponent_MIL,opponent_MIN,opponent_NOP,opponent_NYK,opponent_OKC,opponent_ORL,opponent_PHI,opponent_PHX,opponent_POR,opponent_SAC,opponent_SAS,opponent_TOR,opponent_UTA,opponent_WAS,PTS,POSSESION_Home,POSSESION_Away,PACE,tot_FGM,tot_FGA,tot_3PM,tot_3PA,tot_FTM,tot_FTA,tot_REB,tot_DREB,tot_OREB,tot_STL,tot_AST,tot_TOV,tot_BLK,tot_PF,diff_FGM,diff_FGA,diff_3PM,diff_3PA,diff_FTM,diff_FTA,diff_REB,diff_DREB,diff_OREB,diff_STL,diff_AST,diff_TOV,diff_BLK,diff_PF,TOVg%_Home,TOVg%_Away,REBg%_Home,REBg%_Away,DREBg%_Home,DREBg%_Away,OREBg%_Home,OREBg%_Away,2PM_Home,2PM_Away,2PM%_Home,2PM%_Away,3PM%_Home,3PM%_Away,FTM%_Home,FTM%_Away,STLg%_Home,STLg%_Away,BLKg%_Home,BLKg%_Away,STL/TOV_Home,STL/TOV_Away,OFF_RTG_Home,OFF_RTG_Away,DEF_RTG_Home,DEF_RTG_Away,NET_RTG_Home,NET_RTG_Away,TSA_Home,TSA_Away,TS%_Home,TS%_Away,eFG%_Home,eFG%_Away,AST%_Home,AST%_Away,BLK%_Home,BLK%_Away,DREB%_Home,DREB%_Away,OREB%_Home,OREB%_Away,STL%_Home,STL%_Away,TOV%_Home,TOV%_Away,FF_Home,FF_Away,GS_Home,GS_Away
129,2022-23,IND,2022-10-26T00:00:00,L,24.0,20,47,0.426,8,20,0.400,9,9,1.000,4,12,16,16,6.0,2,5,2,9,10,57,-19.0,1,66,66,66,1,72,43,93,30,26,45,42,85,1,83,117,118,18,34,103,8,31,39,56,54,123,CHI,Away,46.09,2022-23,CHI,2022-10-26T00:00:00,W,24.0,26,44,0.591,9,16,0.563,15,16,0.938,5,19,24,19,5.0,3,2,5,10,9,76,19.0,1,1,1,1,1,11,78,6,13,75,9,5,17,11,61,31,44,5,23,67,71,115,59,73,2,7,IND,Home,76.96,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,50.8416,48.9984,49.9200,46,91,17,36,24,25,40,31,9,5,35,11.0,7,19,-6,3,-1,4,-6,-7,-8,-7,-1,-1,-3,1.0,3,-1,0.545455,0.454545,0.400000,0.600000,0.387097,0.612903,0.444444,0.555556,12,17,0.421053,0.447368,0.421053,0.355263,0.157895,0.197368,0.400000,0.600000,0.714286,0.285714,0.333333,0.600000,112.112915,155.107106,149.483887,116.330329,0.750000,1.333333,50.96,51.04,0.559262,0.744514,1.200000,1.173077,80.000000,73.076923,17.857143,7.407407,70.588235,82.608696,17.391304,29.411765,4.081766,5.900680,10.533708,8.922198,6.463427,7.640480,42.1,71.7
119,2022-23,HOU,2022-10-26T00:00:00,L,24.0,20,46,0.435,7,19,0.368,2,4,0.500,6,12,18,10,12.0,4,1,1,12,10,49,-12.0

## Prediction Models

In [5]:
features = Tools.load_pickle('./ml_models/features_for_prediction_new')

mlp_model = load_model('keras_model.h5')

scaler = Tools.load_pickle('keras_scaler')

kmeans = Tools.load_pickle('keras_clusterer')

### Sanity check

In [6]:
df = data[features]

assert not(df.isnull().values.any()), 'Nan Values in data'

case = np.isinf(df).values.sum()

assert case is not None, 'Inf Values in data'

## Predictions

In [7]:
mapp = {
    'W': 1,
    'L': 0
}

imap = {
        1: 'W',
        0: 'L'
        }

In [8]:
X = np.array(df)
X = scaler.transform(X)
y_val = np.array(data['W/L_Home'].map(mapp))

In [9]:
y_proba_mlp = mlp_model.predict(X)

cond = y_proba_mlp >= 0.5

y_pred_mlp = np.where(cond, 1, 0)

In [10]:
print(f'\nf1_score nn: {Tools.f1_eval(y_pred_mlp, y_val)}\n')
cm = confusion_matrix(y_val, y_pred_mlp)
print(cm)


f1_score nn: 0.7580645161290323

[[53 12]
 [18 47]]


## Post Processing

In [11]:
data['nn_prob'] = y_proba_mlp

data['nn_pred'] = y_pred_mlp

data['nn_pred'] = data['nn_pred'].map(imap)

data['clusters'] = kmeans.predict(np.array(data['nn_prob']).reshape(-1, 1))

In [12]:
past_data = pd.read_csv('clusters_accuracy.csv', index_col=0)
coefs = past_data.groupby('clusters').head(1)
coefs.set_index('clusters', drop=True, inplace=True)
coefs = coefs.to_dict()
acc_map = coefs['Accuracy']

In [13]:
data['Accuracy'] = data['clusters'].map(acc_map)

data['Bet'] = (data.nn_pred == 'L') & (data.Accuracy > 0.75)

In [14]:
matches_betted = data[data.Bet == True].shape[0]
total_matches = data.shape[0] 
correct_predictions = data[(data.Bet == True) & (data.nn_pred == data['W/L_Home'])].shape[0]

In [15]:
print(f'Total matches: {total_matches/2}')
print(f'Matches betted: {matches_betted}')
print(f'frequency {matches_betted/total_matches*2}')
print(f'Correct predictions: {correct_predictions}')
print(f'Accuracy: {correct_predictions/matches_betted}')

Total matches: 65.0
Matches betted: 29
frequency 0.4461538461538462
Correct predictions: 27
Accuracy: 0.9310344827586207


## Profit-Loss

In [16]:
av_coef = 1.3
av_bet = 2

p_l = (correct_predictions*(av_coef - 1) - (matches_betted-correct_predictions))*av_bet
print(f'Total Profit: {round(p_l, 2)} euros')
print(f'Theoritical Money betted: {round(matches_betted*av_bet, 2)} euros')
print(f'Actual Money Betted: {round(((matches_betted-correct_predictions)+1)*av_bet, 2)} euros')

Total Profit: 12.2 euros
Theoritical Money betted: 58 euros
Actual Money Betted: 6 euros


In [17]:
data

,SEASON_YEAR_Home,TEAM_Home,DATE_Home,W/L_Home,MIN_Home,FGM_Home,FGA_Home,FG%_Home,3PM_Home,3PA_Home,3P%_Home,FTM_Home,FTA_Home,FT%_Home,OREB_Home,DREB_Home,REB_Home,AST_Home,TOV_Home,STL_Home,BLK_Home,BLKA_Home,PF_Home,PFD_Home,PTS_Home,+/-_Home,GP_RANK_Home,W_RANK_Home,L_RANK_Home,W_PCT_RANK_Home,MIN_RANK_Home,FGM_RANK_Home,FGA_RANK_Home,FG_PCT_RANK_Home,FG3M_RANK_Home,FG3A_RANK_Home,FG3_PCT_RANK_Home,FTM_RANK_Home,FTA_RANK_Home,FT_PCT_RANK_Home,OREB_RANK_Home,DREB_RANK_Home,REB_RANK_Home,AST_RANK_Home,TOV_RANK_Home,STL_RANK_Home,BLK_RANK_Home,BLKA_RANK_Home,PF_RANK_Home,PFD_RANK_Home,PTS_RANK_Home,PLUS_MINUS_RANK_Home,OPPONENT_Home,VENUE_Home,VALUE_Home,SEASON_YEAR_Away,TEAM_Away,DATE_Away,W/L_Away,MIN_Away,FGM_Away,FGA_Away,FG%_Away,3PM_Away,3PA_Away,3P%_Away,FTM_Away,FTA_Away,FT%_Away,OREB_Away,DREB_Away,REB_Away,AST_Away,TOV_Away,STL_Away,BLK_Away,BLKA_Away,PF_Away,PFD_Away,PTS_Away,+/-_Away,GP_RANK_Away,W_RANK_Away,L_RANK_Away,W_PCT_RANK_Away,MIN_RANK_Away,FGM_RANK_Away,FGA_RANK_Away,FG_PCT_RANK_Away,FG3M_RANK_Away,FG3A_RANK_Away,FG3_PCT_RANK_Away,FTM_RANK_Away,FTA_RANK_Away,FT_PCT_RANK_Away,OREB_RANK_Away,DREB_RANK_Away,REB_RANK_Away,AST_RANK_Away,TOV_RANK_Away,STL_RANK_Away,BLK_RANK_Away,BLKA_RANK_Away,PF_RANK_Away,PFD_RANK_Away,PTS_RANK_Away,PLUS_MINUS_RANK_Away,OPPONENT_Away,VENUE_Away,VALUE_Away,venue_Away,venue_Home,team_ATL,team_BKN,team_BOS,team_CHA,team_CHI,team_CLE,team_DAL,team_DEN,team_DET,team_GSW,team_HOU,team_IND,team_LAC,team_LAL,team_MEM,team_MIA,team_MIL,team_MIN,team_NOP,team_NYK,team_OKC,team_ORL,team_PHI,team_PHX,team_POR,team_SAC,team_SAS,team_TOR,team_UTA,team_WAS,opponent_ATL,opponent_BKN,opponent_BOS,opponent_CHA,opponent_CHI,opponent_CLE,opponent_DAL,opponent_DEN,opponent_DET,opponent_GSW,opponent_HOU,opponent_IND,opponent_LAC,opponent_LAL,opponent_MEM,opponent_MIA,opponent_MIL,opponent_MIN,opponent_NOP,opponent_NYK,opponent_OKC,opponent_ORL,opponent_PHI,opponent_PHX,opponent_POR,opponent_SAC,opponent_SAS,opponent_TOR,opponent_UTA,opponent_WAS,PTS,POSSESION_Home,POSSESION_Away,PACE,tot_FGM,tot_FGA,tot_3PM,tot_3PA,tot_FTM,tot_FTA,tot_REB,tot_DREB,tot_OREB,tot_STL,tot_AST,tot_TOV,tot_BLK,tot_PF,diff_FGM,diff_FGA,diff_3PM,diff_3PA,diff_FTM,diff_FTA,diff_REB,diff_DREB,diff_OREB,diff_STL,diff_AST,diff_TOV,diff_BLK,diff_PF,TOVg%_Home,TOVg%_Away,REBg%_Home,REBg%_Away,DREBg%_Home,DREBg%_Away,OREBg%_Home,OREBg%_Away,2PM_Home,2PM_Away,2PM%_Home,2PM%_Away,3PM%_Home,3PM%_Away,FTM%_Home,FTM%_Away,STLg%_Home,STLg%_Away,BLKg%_Home,BLKg%_Away,STL/TOV_Home,STL/TOV_Away,OFF_RTG_Home,OFF_RTG_Away,DEF_RTG_Home,DEF_RTG_Away,NET_RTG_Home,NET_RTG_Away,TSA_Home,TSA_Away,TS%_Home,TS%_Away,eFG%_Home,eFG%_Away,AST%_Home,AST%_Away,BLK%_Home,BLK%_Away,DREB%_Home,DREB%_Away,OREB%_Home,OREB%_Away,STL%_Home,STL%_Away,TOV%_Home,TOV%_Away,FF_Home,FF_Away,GS_Home,GS_Away,nn_prob,nn_pred,clusters,Accuracy,Bet
129,2022-23,IND,2022-10-26T00:00:00,L,24.0,20,47,0.426,8,20,0.400,9,9,1.000,4,12,16,16,6.0,2,5,2,9,10,57,-19.0,1,66,66,66,1,72,43,93,30,26,45,42,85,1,83,117,118,18,34,103,8,31,39,56,54,123,CHI,Away,46.09,2022-23,CHI,2022-10-26T00:00:00,W,24.0,26,44,0.591,9,16,0.563,15,16,0.938,5,19,24,19,5.0,3,2,5,10,9,76,19.0,1,1,1,1,1,11,78,6,13,75,9,5,17,11,61,31,44,5,23,67,71,115,59,73,2,7,IND,Home,76.96,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,50.8416,48.9984,49.9200,46,91,17,36,24,25,40,31,9,5,35,11.0,7,19,-6,3,-1,4,-6,-7,-8,-7,-1,-1,-3,1.0,3,-1,0.545455,0.454545,0.400000,0.600000,0.387097,0.612903,0.444444,0.555556,12,17,0.421053,0.447368,0.421053,0.355263,0.157895,0.197368,0.400000,0.600000,0.714286,0.285714,0.333333,0.600000,112.112915,155.107106,149.483887,116.330329,0.750000,1.333333,50.96,51.04,0.559262,0.744514,1.200000,1.173077,80.000000,73.076923,17.857143,7.407407,70.588235,82.608696,17.391304,29.411765,4.081766,5.900680,10.533708,8.922198,6.463427,7.640480,42.1,71.7,0.264781,L,48,0.886076,True
119,2022-23,HOU,2022-10-26T00:00:00,L,24.0,20,